# Imports

In [1]:
import pandas as pd
import numpy as np
import pathlib
from pathlib import Path
import os
import duckdb

pd.set_option('display.max_columns', None)

# Violência contra mulheres

In [2]:
BASE_PATH = Path("/home/matsa/Documents/Faculdade/SAD/tp01/dados_violencia_contra_mulher_SES/")

In [3]:
def concat_datasets(datasets_path:pathlib.Path, output_filename):
    with duckdb.connect("db.db") as con:
        output_file = Path(datasets_path / f"{output_filename}.csv")
        
        query = f"""
        COPY (
            SELECT * 
            FROM read_csv_auto('{datasets_path}/*.csv', union_by_name=True)
        )
        TO '{output_file}' WITH (FORMAT CSV, HEADER TRUE);
        """

        con.execute(query)
        print(f"Files of {datasets_path} concatenated and saved to {output_file}")

concat_datasets(BASE_PATH, "dados_violencia_mulheres_ses_concat")

Files of /home/matsa/Documents/Faculdade/SAD/tp01/dados_violencia_contra_mulher_SES concatenated and saved to /home/matsa/Documents/Faculdade/SAD/tp01/dados_violencia_contra_mulher_SES/dados_violencia_mulheres_ses_concat.csv


In [3]:
df_concat = pd.read_csv("./dados_violencia_contra_mulher_SES/dados_violencia_mulheres_ses_concat.csv", sep=",")

display(df_concat.head())
print(f"Shape: {df_concat.shape}")

,dt_notific,dt_nasc,nu_idade_n,cs_sexo,cs_raca,id_mn_resi,local_ocor,out_vezes,les_autop,viol_fisic,viol_psico,viol_sexu,num_envolv,autor_sexo,orient_sex,ident_gen
0,2010-08-03,1937-11-02,72,Feminino,Parda,Governador Valadares,Residencia,Sim,Não,Sim,Sim,Não,Um,Masculino,NaN,NaN
1,2010-02-07,1992-04-30,17,Feminino,Parda,Montes Claros,Residencia,Ignorado,Não,Sim,Sim,NaN,Um,Masculino,NaN,NaN
2,2010-06-28,2003-04-07,7,Feminino,Ignorado,Governador Valadares,Ignorado,Ignorado,Não,Não,Sim,Sim,Um,Masculino,NaN,NaN
3,2010-01-07,1963-05-07,46,Feminino,Parda,São José do Goiabal,Residencia,Ignorado,NaN,Não,Não,Não,Um,Feminino,NaN,NaN
4,2010-05-20,1917-08-01,92,Feminino,Branca,Jaboticatubas,Residencia,Sim,NaN,Não,Não,Não,Dois ou mais,Masculino,NaN,NaN


Shape: (472046, 16)


In [4]:
df_concat["cs_sexo"].value_counts()

cs_sexo
Feminino    472046
Name: count, dtype: int64

In [5]:
df_concat.dtypes

dt_notific    object
dt_nasc       object
nu_idade_n    object
cs_sexo       object
cs_raca       object
id_mn_resi    object
local_ocor    object
out_vezes     object
les_autop     object
viol_fisic    object
viol_psico    object
viol_sexu     object
num_envolv    object
autor_sexo    object
orient_sex    object
ident_gen     object
dtype: object

In [6]:
df_concat.isna().sum()

dt_notific        0
dt_nasc        6068
nu_idade_n       36
cs_sexo           0
cs_raca        5982
id_mn_resi     1265
local_ocor    11717
out_vezes     15529
les_autop     15789
viol_fisic    12715
viol_psico    17370
viol_sexu     17863
num_envolv    18096
autor_sexo    13492
orient_sex    79176
ident_gen     79176
dtype: int64

In [7]:
duplicated_rows = df_concat[df_concat.duplicated(keep=False)]

duplicated_count = duplicated_rows.groupby(
    list(df_concat.columns)
).size().reset_index(name="duplicates_count")

display(duplicated_count.head(16))
print("Duplicates count: ", duplicated_count.shape[0], "\n")
print(duplicated_count["duplicates_count"].value_counts())

,dt_notific,dt_nasc,nu_idade_n,cs_sexo,cs_raca,id_mn_resi,local_ocor,out_vezes,les_autop,viol_fisic,viol_psico,viol_sexu,num_envolv,autor_sexo,orient_sex,ident_gen,duplicates_count
0,2014-07-11,1991-03-04,"23,0",Feminino,Parda,Prata,Residencia,Sim,Não,Sim,Sim,Não,Um,Masculino,Heterossexual,Não se aplica,2
1,2014-10-14,1962-10-10,"51,0",Feminino,Ignorado,Congonhas,Ignorado,Ignorado,Ignorado,Sim,Não,Não,Ignorado,Ignorado,Ignorado,Ignorado,2
2,2014-10-19,1980-04-14,"34,0",Feminino,Parda,Uberaba,Residencia,Sim,Não,Sim,Não,Não,Um,Masculino,Ignorado,Ignorado,2
3,2014-10-25,1993-03-12,"21,0",Feminino,Parda,Itabira,Via pública,Não,Não,Não,Não,Sim,Ignorado,Masculino,Ignorado,Ignorado,2
4,2014-10-28,2013-05-10,"1,0",Feminino,Parda,Pompéu,Residencia,Não,Não,Sim,Não,Não,Um,Masculino,Não se aplica,Não se aplica,2
5,2014-11-08,1994-08-03,"20,0",Feminino,Branca,Urucânia,Residencia,Sim,Sim,Não,Não,Não,Um,Feminino,Ignorado,Ignorado,2
6,2014-11-19,1976-03-16,"38,0",Feminino,Branca,Nova Serrana,Residencia,Não,Não,Sim,Sim,Sim,Um,Masculino,Heterossexual,Não se aplica,2
7,2014-11-21,1996-02-18,"18,0",Feminino,Preta,Leandro Ferreira,Residencia,Sim,Não,Sim,Não,Não,Um,Masculino,Heterossexual,Não se aplica,2
8,2014-12-18,1962-07-08,"52,0",Feminino,Branca,Santa Rita do Itueto,Residencia,Sim,Não,Sim,Sim,Não,Um,Masculino,Heterossexual,Não se aplica,2
9,2014-12-31,1984-11-27,"30,0",Feminino,Branca,Nova Resende,Residencia,Sim,Sim,Sim,Não,Não,Um,Feminino,Heterossexual,Não se aplica,2


Duplicates count:  651 

duplicates_count
2    639
3     12
Name: count, dtype: int64


In [8]:
df_concat = df_concat.drop_duplicates()

In [9]:
df_concat = df_concat.rename({"dt_notific": "dt_notificacao", 
                  "dt_nasc": "dt_nascimento_vitima", 
                  "nu_idade_n": "idade_vitima", 
                  "cs_raca": "raca_vitima", 
                  "id_mn_resi": "municipio_residencia", 
                  "local_ocor": "local_ocorrencia", 
                  "out_vezes": "outras_vezes", 
                  "les_autop": "lesao_autoprovocada", 
                  "viol_fisic": "viol_fisica", 
                  "viol_psico": "viol_psicologica", 
                  "viol_sexu": "viol_sexual", 
                  "num_envolv": "num_envolvidos",
                  "orient_sex": "orient_sex_vitima", 
                  "ident_gen": "ident_gen_vitima"}, axis=1)

def classify_violence(row):
    types = []

    if row["viol_fisica"] == "Sim":
        types.append("Física")
    if row["viol_psicologica"] == "Sim":
        types.append("Psicológica")
    if row["viol_sexual"] == "Sim":
        types.append("Sexual")

    return " | ".join(types) if types else "Não preenchido"
    
df_concat["tipo_violencia"] = df_concat.apply(classify_violence, axis=1)


##########################
### filling NaN values ###
##########################

### numeric columns ###

# converting datas
df_concat["dt_notificacao"] = pd.to_datetime(df_concat["dt_notificacao"], errors="coerce")
df_concat["dt_nascimento_vitima"] = pd.to_datetime(df_concat["dt_nascimento_vitima"], errors="coerce")

# split catches only the number
df_concat["idade_vitima"] = df_concat["idade_vitima"].astype(str).str.split(",", expand=True)[0]
df_concat["idade_vitima"] = pd.to_numeric(df_concat["idade_vitima"], errors="coerce")
df_concat["idade_vitima"] = df_concat["idade_vitima"].fillna(-1).astype("int16")

# returns the days quantity between this dates and catches the Integer part  (//)
df_concat["idade_calc"] = (df_concat["dt_notificacao"] - df_concat["dt_nascimento_vitima"]).dt.days // 365
# combines the age of the original column and the calc age. If still NaN, fill with -1
df_concat["idade_vitima"] = (
    df_concat["idade_calc"]
        .fillna(df_concat["idade_vitima"])
        .fillna(-1)
        .astype("int16")
)

### object columns ###

fillna_object_columns = ["raca_vitima", "municipio_residencia", "local_ocorrencia", "outras_vezes", "num_envolvidos", "lesao_autoprovocada", "autor_sexo", "orient_sex_vitima", "ident_gen_vitima"]

for col in fillna_object_columns:
    df_concat[col] = df_concat[col].fillna("Não preenchido")


#######################
### changing dtypes ###
#######################

df_concat = df_concat.astype({
    "dt_nascimento_vitima": "datetime64[ns]",
    "municipio_residencia": "string", 
    "idade_vitima": "int16", 
    "raca_vitima": "string", 
    "orient_sex_vitima": "string", 
    "ident_gen_vitima": "string", 
    "dt_notificacao": "datetime64[ns]", 
    "local_ocorrencia": "string", 
    "num_envolvidos": "string", 
    "tipo_violencia": "string",
    "lesao_autoprovocada": "string", 
    "outras_vezes": "string"
})


df_concat = df_concat[["dt_nascimento_vitima", "municipio_residencia", "idade_vitima", "raca_vitima", "orient_sex_vitima", "ident_gen_vitima", "dt_notificacao", "local_ocorrencia", "num_envolvidos", "tipo_violencia", "lesao_autoprovocada", "outras_vezes", "autor_sexo"]]

display(df_concat.head(20))
print("Shape: ", df_concat.shape)

,dt_nascimento_vitima,municipio_residencia,idade_vitima,raca_vitima,orient_sex_vitima,ident_gen_vitima,dt_notificacao,local_ocorrencia,num_envolvidos,tipo_violencia,lesao_autoprovocada,outras_vezes,autor_sexo
0,1937-11-02,Governador Valadares,72,Parda,Não preenchido,Não preenchido,2010-08-03,Residencia,Um,Física | Psicológica,Não,Sim,Masculino
1,1992-04-30,Montes Claros,17,Parda,Não preenchido,Não preenchido,2010-02-07,Residencia,Um,Física | Psicológica,Não,Ignorado,Masculino
2,2003-04-07,Governador Valadares,7,Ignorado,Não preenchido,Não preenchido,2010-06-28,Ignorado,Um,Psicológica | Sexual,Não,Ignorado,Masculino
3,1963-05-07,São José do Goiabal,46,Parda,Não preenchido,Não preenchido,2010-01-07,Residencia,Um,Não preenchido,Não preenchido,Ignorado,Feminino
4,1917-08-01,Jaboticatubas,92,Branca,Não preenchido,Não preenchido,2010-05-20,Residencia,Dois ou mais,Não preenchido,Não preenchido,Sim,Masculino
5,2010-01-19,Pouso Alegre,0,Branca,Não preenchido,Não preenchido,2010-07-09,Ignorado,Um,Física,Não,Ignorado,Feminino
6,1999-06-06,Bambuí,10,Branca,Não preenchido,Não preenchido,2010-02-09,Residencia,Um,Física,Não,Ignorado,Masculino
7,1955-01-30,Contagem,55,Ignorado,Não preenchido,Não preenchido,2010-01-23,Residencia,Ignorado,Física,Não,Ignorado,Ignorado
8,1980-09-08,Patos de Minas,29,Branca,Não preenchido,Não preenchido,2010-03-01,Ignorado,Ignorado,Física,Não preenchido,Não,Ignorado
9,1985-08-25,Várzea da Palma,24,Parda,Não preenchido,Não preenchido,2010-02-07,Via pública,Um,Física,Não,Não,Feminino


Shape:  (471260, 13)


In [10]:
df_concat.isna().sum()

dt_nascimento_vitima    6042
municipio_residencia       0
idade_vitima               0
raca_vitima                0
orient_sex_vitima          0
ident_gen_vitima           0
dt_notificacao             0
local_ocorrencia           0
num_envolvidos             0
tipo_violencia             0
lesao_autoprovocada        0
outras_vezes               0
autor_sexo                 0
dtype: int64

In [11]:
(df_concat["idade_vitima"] == -1).sum()

np.int64(38)

In [12]:
df_concat.index = df_concat.index + 1
df_concat.index.name = "id"
df_concat.to_csv("./dados_violencia_contra_mulher_SES/dados_violencia_mulheres_ses_concat_limpo.csv", index=True)

In [13]:
df = pd.read_csv(BASE_PATH / "dados_violencia_mulheres_ses_concat_limpo.csv")

display(df.head())
print("Shape", df.shape)

,id,dt_nascimento_vitima,municipio_residencia,idade_vitima,raca_vitima,orient_sex_vitima,ident_gen_vitima,dt_notificacao,local_ocorrencia,num_envolvidos,tipo_violencia,lesao_autoprovocada,outras_vezes,autor_sexo
0,1,1937-11-02,Governador Valadares,72,Parda,Não preenchido,Não preenchido,2010-08-03,Residencia,Um,Física | Psicológica,Não,Sim,Masculino
1,2,1992-04-30,Montes Claros,17,Parda,Não preenchido,Não preenchido,2010-02-07,Residencia,Um,Física | Psicológica,Não,Ignorado,Masculino
2,3,2003-04-07,Governador Valadares,7,Ignorado,Não preenchido,Não preenchido,2010-06-28,Ignorado,Um,Psicológica | Sexual,Não,Ignorado,Masculino
3,4,1963-05-07,São José do Goiabal,46,Parda,Não preenchido,Não preenchido,2010-01-07,Residencia,Um,Não preenchido,Não preenchido,Ignorado,Feminino
4,5,1917-08-01,Jaboticatubas,92,Branca,Não preenchido,Não preenchido,2010-05-20,Residencia,Dois ou mais,Não preenchido,Não preenchido,Sim,Masculino


Shape (471260, 14)


In [15]:
df.columns

Index(['id', 'dt_nascimento_vitima', 'municipio_residencia', 'idade_vitima',
       'raca_vitima', 'orient_sex_vitima', 'ident_gen_vitima',
       'dt_notificacao', 'local_ocorrencia', 'num_envolvidos',
       'tipo_violencia', 'lesao_autoprovocada', 'outras_vezes', 'autor_sexo'],
      dtype='object')

# --------------------------------------------------------

In [4]:
test = pd.read_csv(BASE_PATH / "dados_violencia_mulheres_ses_concat.csv")
test

,dt_notific,dt_nasc,nu_idade_n,cs_sexo,cs_raca,id_mn_resi,local_ocor,out_vezes,les_autop,viol_fisic,viol_psico,viol_sexu,num_envolv,autor_sexo,orient_sex,ident_gen
0,2010-08-03,1937-11-02,72,Feminino,Parda,Governador Valadares,Residencia,Sim,Não,Sim,Sim,Não,Um,Masculino,NaN,NaN
1,2010-02-07,1992-04-30,17,Feminino,Parda,Montes Claros,Residencia,Ignorado,Não,Sim,Sim,NaN,Um,Masculino,NaN,NaN
2,2010-06-28,2003-04-07,7,Feminino,Ignorado,Governador Valadares,Ignorado,Ignorado,Não,Não,Sim,Sim,Um,Masculino,NaN,NaN
3,2010-01-07,1963-05-07,46,Feminino,Parda,São José do Goiabal,Residencia,Ignorado,NaN,Não,Não,Não,Um,Feminino,NaN,NaN
4,2010-05-20,1917-08-01,92,Feminino,Branca,Jaboticatubas,Residencia,Sim,NaN,Não,Não,Não,Dois ou mais,Masculino,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472041,2025-10-31,2002-01-15,"23,0",Feminino,Parda,São Gotardo,Residencia,Não,Sim,Não,Não,Não,Um,Feminino,Heterossexual,Não se aplica
472042,2025-11-03,2009-02-11,"16,0",Feminino,Indígena,Patos de Minas,Residencia,Não,Sim,Sim,Não,Não,Um,Feminino,Heterossexual,Não se aplica
472043,2025-10-23,1997-02-28,"28,0",Feminino,Parda,Patos de Minas,Residencia,Sim,Sim,Sim,Não,Não,Um,Feminino,Heterossexual,Não se aplica
472044,2025-10-23,1956-01-08,"69,0",Feminino,Parda,Lagoa Formosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test["dt_notific"].min()

'2010-01-01'

In [7]:
test["dt_notific"].max()

'2025-11-07'

In [4]:
test.isna().sum()

dt_notific        0
dt_nasc        6068
nu_idade_n       36
cs_sexo           0
cs_raca        5982
id_mn_resi     1265
local_ocor    11717
out_vezes     15529
les_autop     15789
viol_fisic    12715
viol_psico    17370
viol_sexu     17863
num_envolv    18096
autor_sexo    13492
orient_sex    79176
ident_gen     79176
dtype: int64